<a href="https://colab.research.google.com/github/sumit298/ML-Journal/blob/main/Llama_2_7b_chat_hf_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install peft
!pip install acclerate
!pip install -U bitsandbytes
!pip install transformers
!pip install datasets

ERROR: Could not find a version that satisfies the requirement acclerate (from versions: none)
ERROR: No matching distribution found for acclerate


In [ ]:
!pip install GPUtil

In [ ]:
import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
  print("GPU is available")
else:
  print("GPU is not available, use CPU instead")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset

if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()

In [ ]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read)

In [ ]:
base_model_id = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
!git clone https://github.com/poloclub/Fine-tuning-LLMs.git

Cloning into 'Fine-tuning-LLMs'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 14), reused 29 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 9.34 MiB | 34.28 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [ ]:
train_dataset = load_dataset("text", data_files={
    "train": ["/content/Fine-tuning-LLMs/data/hawaii_wf_1.txt", "/content/Fine-tuning-LLMs/data/hawaii_wf_2.txt", "/content/Fine-tuning-LLMs/data/hawaii_wf_3.txt", "/content/Fine-tuning-LLMs/data/hawaii_wf_4.txt" ]}
    , split="train"
)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_dataset["text"][10]

'During the hours of August 8, 2023, Maui became the stage for the most tragic natural disaster in state history and the deadliest fire in modem American history. '

In [ ]:
from transformers import LlamaTokenizer

# Load the Llama tokenizer
tokenizer = LlamaTokenizer.from_pretrained(
    base_model_id,   # 📌 Model identifier (e.g., "meta-llama/Llama-2-7b-chat-hf")

    use_fast=False,  # ⚡ False = Use Python-based tokenizer (more stable for Llama)
                     #    True  = Use Rust-based tokenizer (faster, but may have issues)

    trust_remote_code=True,  # 🔐 Enables loading of custom tokenizer logic from Hugging Face.
                             # ⚠️ Only use this if you trust the model source!

    add_eos_token=True  # 🏁 Adds an EOS (End-of-Sequence) token to the text automatically.
                        #    Useful for models that require a clear stopping point.
)

# 🔹 Llama models do NOT have a pad token by default.
# 🔹 If there's no pad token, we assign the EOS token as the pad token.
if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})  # 🛑 Sets EOS as PAD token to avoid errors

# ✅ Now, the tokenizer is ready to handle variable-length inputs safely.
print(f"Tokenizer Loaded! PAD Token: {tokenizer.pad_token}, EOS Token: {tokenizer.eos_token}")



tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Tokenizer Loaded! PAD Token: </s>, EOS Token: </s>


In [ ]:
tokenized_train_dataset = []

for phrase in train_dataset:
  tokenized_train_dataset.append(tokenizer(phrase['text']))

In [ ]:
tokenized_train_dataset[8]


{'input_ids': [1, 399, 29909, 2287, 341, 29889, 14861, 3352, 29909, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


## The attention_mask tells the model which tokens to focus on and which to ignore during training or inference.

Let's say we have two sentences:

1. "I love NLP" (3 tokens)
2. "Transformers are amazing!" (4 tokens)


If we send them directly into the model, their lengths don’t match:

```py
Sentence 1: [1, 314, 1562, 2]      # 4 tokens (with EOS)
Sentence 2: [1, 2345, 765, 45678, 2] # 5 tokens (with EOS)
```

```py
{
    'input_ids': [
        [1, 314, 1562, 2, 0, 0],    # "I love NLP" + Padding (0s)
        [1, 2345, 765, 45678, 2]    # "Transformers are amazing!" (No padding needed)
    ],
    'attention_mask': [
        [1, 1, 1, 1, 0, 0],   # ✅ Model attends to the first 4 tokens, ignores the last 2
        [1, 1, 1, 1, 1]       # ✅ No padding, all tokens attended to
    ]
}


## Why Do We Use Padding?

* ✅ Ensures each input has the same shape → Prevents shape mismatch errors.
* ✅ Efficient GPU computation → GPUs process fixed-size tensors faster.
* ✅ Attention mask helps the model ignore padding → Prevents learning from unnecessary [PAD] tokens.

```py
from transformers import LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

sentences = ["I love NLP", "Transformers are amazing!"]

# Tokenizing with automatic padding
encoded = tokenizer(
    sentences,
    padding=True,  # ✅ Pads to the longest sentence
    truncation=True,
    return_tensors="pt"
)

print(encoded)
```

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import LlamaForCausalLM

# Enable gradient checkpointing to reduce memory usage during training
model.gradient_checkpointing_enable()

# Convert model for k-bit (low-precision) training
# This is useful for training large models with limited VRAM
model = prepare_model_for_kbit_training(model)

# 🔹 LoRA Configuration (Low-Rank Adaptation)
config = LoraConfig(
    r=8,  # Rank of the low-rank matrices (smaller r → less compute, larger r → better learning)
    lora_alpha=32,  # Scaling factor for LoRA weights (higher → stronger impact on adaptation)

    # Define which model layers to apply LoRA (these are key transformer layers)
    target_modules=[
        "q_proj",   # Query projection
        "k_proj",   # Key projection
        "v_proj",   # Value projection
        "o_proj",   # Output projection
        "gate_proj", # Gated linear unit (GLU) projection
        "up_proj",  # Feedforward network (FFN) up-projection
        "down_proj" # FFN down-projection
    ],

    bias="none",  # "none" → No additional bias parameters (for efficiency)

    lora_dropout=0.05,  # Dropout for LoRA layers (helps with generalization)

    task_type="CAUSAL_LM"  # Task type → Causal Language Modeling (predicts next token in a sequence)
)

# 🔹 Apply LoRA to the model
model = get_peft_model(model, config)


### 🛠️ Quick Summary of Parameters

| Parameter                           | Purpose |
|-------------------------------------|---------|
| `gradient_checkpointing_enable()`   | Reduces memory usage by recomputing activations during backward pass. |
| `prepare_model_for_kbit_training(model)` | Converts model for low-bit (quantized) training. |
| `r=8`                               | Controls LoRA's rank (higher = better performance but more compute). |
| `lora_alpha=32`                     | Scaling factor (higher = stronger adaptation). |
| `target_modules`                     | Specifies which layers get LoRA modifications. |
| `bias="none"`                        | No extra biases (reduces model size). |
| `lora_dropout=0.05`                  | Adds dropout to prevent overfitting. |
| `task_type="CAUSAL_LM"`               | Defines the task (language modeling). |


In [ ]:
# Import necessary libraries
import transformers

# Initialize the Trainer class with the model and training configurations
trainer = transformers.Trainer(
    model=model,  # The fine-tuning model
    train_dataset=tokenized_train_dataset,  # Tokenized dataset used for training

    # Training arguments
    args=transformers.TrainingArguments(
        output_dir="./finetunedModel",  # Where to save the model checkpoints
        per_device_train_batch_size=2,  # Batch size per GPU/TPU core
        gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps before updating model
        num_train_epochs=3,  # Number of full passes through the dataset
        learning_rate=2e-4,  # Learning rate for the optimizer
        max_steps=1000,  # Maximum number of training steps (overrides num_train_epochs if reached earlier)
        bf16=False,  # Use Brain Floating Point 16 (set to False if not supported)
        optim="paged_adamw_8bit",  # Optimizer type (8-bit AdamW to save memory)
        logging_dir="./log",  # Directory to store training logs
        save_strategy="epoch",  # Save model after each epoch
        save_steps=50,  # Save a checkpoint every 50 steps
        logging_steps=10  # Log training metrics every 10 steps
    ),

    # Data collator: helps in preparing batches of data for training
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    # mlm=False → This means we are doing causal language modeling (next-word prediction) instead of masked language modeling
)

# Disable caching during training (helps with fine-tuning stability)
model.config.use_cache = False

# Start training
trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sumitssss298 (sumitssss298-nirma-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.599900
20,3.083800
30,2.666400
40,2.794700
50,2.729400
60,2.536300
70,2.264400
80,2.380300
90,2.447800
100,2.352800


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

# 🛠️ Trainer Arguments - Quick Summary

| **Parameter**                        | **Purpose** |
|--------------------------------------|-------------|
| **`output_dir="/finetunedModel"`**       | 📂 Directory to save the fine-tuned model checkpoints. |
| **`per_device_train_batch_size=2`**      | 🎯 Number of samples per batch for each GPU (low to fit memory). |
| **`gradient_accumulation_steps=2`**      | 🔄 Gradients accumulate over 2 steps before updating weights (simulates larger batch size). |
| **`num_train_epochs=3`**                 | 🔁 Number of times the dataset is passed through the model. |
| **`learning_rate=2e-4`**                 | ⚡ Determines how much to adjust weights during training. |
| **`max_steps=1500`**                     | ⏳ Stops training after 1500 steps (overrides epochs if reached first). |
| **`bf16=False`**                         | 🚀 Whether to use bfloat16 precision (set to `False` if unsupported). |
| **`optim="paged_adamw_8bit"`**           | 🧠 Uses 8-bit AdamW optimizer to reduce memory usage. |
| **`logging_dir="./log"`**                | 📝 Saves training logs for tracking performance. |
| **`save_strategy="epoch"`**              | 💾 Saves the model at the end of each epoch. |
| **`save_steps=50`**                      | 📌 Saves a checkpoint every 50 steps. |
| **`logging_steps=10`**                    | 📊 Logs training status every 10 steps. |
| **`data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)`** | ✅ Ensures correct batching and tokenization (**mlm=False** means **causal LM**). |


## Inferencing is the process of using your fine-tuned model to generate predictions or responses based on new input data.

For LLaMA-based models, inferencing means:

- Giving the model a prompt (input text).

- Getting a generated response (output text).

In [ ]:
# Inference
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from peft import PeftModel
model_id = "meta-llama/Llama-2-7b-chat-hf"

nf4Config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = LlamaTokenizer.from_pretrained(model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=nf4Config,
    device_model="auto",
    trust_remote_code=True,
    use_auth_token=True
  )

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(model_id, use_fast=False, trust_remote_code=True)

modelFinetuned = PeftModel.from_pretrained(base_model, "./finetunedModel/checkpoint-1000")

In [ ]:
user_question = "When did Hawaii wildfires start"

eval_prompt = f"Question: {user_question} just answer this question accurately and concisely."

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()

with torch.no_grad():
  print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens=1024)[0], torch.cuda.empty_cache))